In [1]:
import os, sys, inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
gparentdir = os.path.dirname(os.path.dirname(currentdir))
sys.path.insert(0, gparentdir)
import scobra
import get_all_functions as gaf
import re
new = scobra.Model("sample/Diel_Leaf_Model_cleaned.xls")
#new = scobra.model(n)

#new=scobra.Model('models/Diel_Leaf_Model_cleaned.xls')

#Maximize Sucrose output (objective)
#vary light (scan)
#allow CO2 and O2, even Proton and Water
#Output maintenance ATP and NADPHox
tx_and_biomass = []
for i in new.Reactions(): 
    if '_tx' in i:
        tx_and_biomass.append(re.search('(.*)_tx', i).group())
    elif '_biomass' in i: 
        tx_and_biomass.append(re.search('(.*)_biomass', i).group())
#tx_and_biomass.append(re.search('(.*)_tx', 'slkdfjd_tx').group())
tx_and_biomass

necessary_rxn= ['ATPase_tx', 'CO2_tx', 'Photon_tx', 'O2_tx','H2O_tx', 'H_tx','NADPHoxc_tx','NADPHoxm_tx','NADPHoxp_tx','Sucrose_tx']
unecessary_rxn= list(set(tx_and_biomass) - set(necessary_rxn))

#blocking _tx and _biomass reactions that are not considered
for i in unecessary_rxn: 
    new.GetReaction(i).lower_bound = 0.0
    new.GetReaction(i).upper_bound = 0.0

#freeing _tx and _biomass reactions that are considered
for i in necessary_rxn: 
    if new.GetReaction(i).lower_bound == 0.0:
        new.GetReaction(i).upper_bound == 1000.0
    else: 
        new.GetReaction(i).lower_bound == 1000.0
        new.GetReaction(i).upper_bound == 1000.0

#setting constraints
new.SetConstraint('ATPase_tx', 90, 90)
new.SetConstraint('NADPHoxc_tx', 10, 10)
new.SetConstraint('NADPHoxm_tx', 10, 10)
new.SetConstraint('NADPHoxp_tx', 10, 10)
#setting constraints for sucrose entering and exiting cytoplasm
new.SetConstraint('Sucrose_ec', -1000, 1000)

#setting objectives
new.SetObjective('Sucrose_tx')
new.SetObjDirec('Min')

#setting constraint and running scan
#result = {}
#for i in range(0,1000,20):
#    new.SetConstraint('Photon_tx', i, i)
#    new.Solve()
#    x = new.GetSol()['Sucrose_tx']
#    result[i] = x

#new.Solve() 
#new.MinFluxSolve()

new.Solve()

cobra/core/reaction.py:394 UserWarning: malformed gene_reaction_rule '(  )' for <Reaction HEXOKINASE_RXN_MANNOSE_c at 0x7fa7b8d1ff50>


optimal


In [2]:
refflux = {'1TRANSKETO_RXN_p': 31.979030144167627,
 '1_PERIOD_10_PERIOD_2_PERIOD_2_RXN_mi': 28.12581913499394,
 '1_PERIOD_18_PERIOD_1_PERIOD_2_RXN_p': 250.00000000000009,
 '2KG_ISOCITRATE_mc': 9.999999999999966,
 '2TRANSKETO_RXN_p': -31.979030144167627,
 'ADENYL_KIN_RXN_c': -42.188728702489634,
 'ADENYL_KIN_RXN_m': 42.18872870248983,
 'ATP_ADP_Pi_pc': 13.617300131062754}

In [3]:
import scobra.analysis.ROOM as room 
room.ROOM(new, refflux)

{'1TRANSKETO_RXN_p': -1.718694149886776,
 '1_PERIOD_10_PERIOD_2_PERIOD_2_RXN_mi': 28.12581913499394,
 '1_PERIOD_18_PERIOD_1_PERIOD_2_RXN_p': 5.624925533182362,
 '1_PERIOD_2_PERIOD_1_PERIOD_9_RXN_c': 6.093768616704414,
 '2KG_ISOCITRATE_mc': 2.9685452162515165,
 '2OXOGLUTARATEDEH_RXN_m': 7.031454783748482,
 '2PGADEHYDRAT_RXN_c': 7.031454783748485,
 '2TRANSKETO_RXN_p': 1.718694149886776,
 '2_PERIOD_7_PERIOD_1_PERIOD_90_RXN_c': -1.9843236710850851,
 '3PGAREARR_RXN_c': 7.031454783748485,
 '3PGA_Pi_pc': 0.9376861670440712,
 '6PFRUCTPHOS_RXN_c': 4.171860904493904,
 '6PGLUCONDEHYDROG_RXN_c': 3.437388299773552,
 '6PGLUCONDEHYDROG_RXN_p': 2.1875372334088192,
 '6PGLUCONOLACT_RXN_c': 3.437388299773552,
 '6PGLUCONOLACT_RXN_p': 2.1875372334088192,
 'ACONITATEDEHYDR_RXN_c': 7.031454783748483,
 'ACONITATEHYDR_RXN_m': 7.031454783748483,
 'ADENYL_KIN_RXN_c': -41.485583224116056,
 'ADENYL_KIN_RXN_m': 41.48558322411607,
 'ATP_ADP_Pi_pc': 6.763204030569581,
 'ATP_AMP_mc': 41.48558322411607,
 'ATPase_tx': 9

In [ ]:
print('Test Completed')